In [35]:
# since we want d = 5 we can just ignore the initial coding about input & calculate p, n_d
d = 5
p,_ = is_prime_power(d, get_data=True)
if p == 2:
    n_d = 6 * d
elif p == 3:
    n_d = 5 * d
else:
    n_d = 4 * d

# i don't know what following things about m_dn, g_dn and max_deg tho. So maybe we can reduce this code like make a easier m_dn or g_dn. not sure if that is possible
R.<x,y,z> = PolynomialRing(ZZ)
m_dn_all = [] # m_dn_all = [2, 4, 6, 8]
g_dn_all = [] # g_dn_all = [1, x^2, x^2 - 1, x^4 - 2*x^2]
for n in range(d,n_d + 1, d):
    m_dn = ceil(sum(euler_phi(2 * n // i) for i in divisors(2 * n // (d * gcd(2,p)))) / 4)
    g_dn = R(chebyshev_U(n * d // p**(valuation(n, p) + 1) - 1, x / 2))
    if g_dn.degree(x) % 2 != 0:
        g_dn = x * g_dn
    m_dn_all.append(m_dn)
    g_dn_all.append(g_dn)
    n = n + d

max_deg = 0 # max_deg = 100
for i in range(len(m_dn_all)):
    max_deg = max(4 * n_d + g_dn_all[i].degree(x) + 2 * m_dn_all[i], max_deg)

reduction_cache = []

# (waiting for Reyna to improve)

# this one is store the function in reduction_cache [n][b]. 
# for example x^ay^b -> 2x^(a-1)y^(b-1)z, so store 2*reduction_cache[n-1][b-1][1] (because get -1 degree) in reduction_cache[n][b][0] (just a little bit like recurcive, so endedup store sth of pure x)
# I just delate the term with k, remain rest the same
def reduce_monomial(n, b, c): # x^(n-b-c), y^(b), z^(c)
    a = n - b - c
    if b == 0 and c == 0: # for b,c = 0 so base case (only have x)
        reduction_cache.append([])
        reduction_cache[n].append([])
        reduction_cache[n][0].append(x**a)
    elif c == 0 and b > 0: # for c = 0, using x^ay^b -> 2x^(a-1)y^(b-1)z
        triple = sorted([a-1, b-1, 1], reverse=True)
        reduction_cache[n].append([])
        reduction_cache[n][b].append(2 * reduction_cache[n-1][triple[1]][triple[2]])
    else: # for all things not equal to 0, using x^ay^bz^c = x^(a-1)y^(b-1)z^(c-1)(x^2+y^2+z^2)
        triples = [sorted([a-1, b+1, c-1], reverse=True),sorted([a-1, b-1, c+1], reverse=True)]
        reduction_cache[n][b].append(reduction_cache[n-1][b-1][c-1] + reduction_cache[n-1][triples[0][1]][triples[0][2]] + reduction_cache[n-1][triples[1][1]][triples[1][2]])

# Following is to fill the reduction_cache
for n in range(max_deg+1):
    for b in range(n//2 + 1):
        for c in range(min(b, n - 2*b) + 1):
            reduce_monomial(n, b, c)
    if n >= 3: # only want c = 0. not sure why he can do that.
        for b in range((n-3)//2 + 1):
            reduction_cache[n-3][b] = reduction_cache[n-3][b][0]
    # this following just check if every degree is filled 
    if n == max_deg:
        print(n)
    else:
        print(f"{n}, ", end='', flush=True)

# start of lloyed chunk==================================================================================

# This function removes small prime powers of an integer
def reduce_int(content):
    if content == 0: #0 has infinite divisibility so we must end if content is 0
        return 0
    for p in prime_range(2 * n_d):
        content = content // p**valuation(content, p)
        #valuation gives the power of said prime in content's prime factorization
    return content 
# essentially, this removes all prime powers for p<2n_d


S.<x,y,z> = PolynomialRing(ZZ)
print(f"\n\nComputing reductions of {sum(m_dn_all)} eigenvectors: ", end='', flush=True)
rows = []
tot = 0
for n in range(d,n_d + 1, d):
# we start with d and make jumps of incriment d until we hit n_d+1
    poly = 0
    for i in range(n+1):
        poly += ((-1)**i * 2 * n * binomial(n + i, n - i) // (n + i)) * (x**2)**(n - i) * (x**2 - 4)**i * y**(2*i)
    poly *= g_dn_all[n // d - 1]
    
    # the above portion uses functions f,g (from 6.18,8.1) (this is line 4 in the paper)
    for m in range(m_dn_all[n // d - 1]):
    # line 6 in the paper, this is the haziest part (uses phi)
    # made to reduce with kappa a variable but for us these should only be constants
    # gives phi(x^{2m}gf) makes a polynomial a_1+...+a_{n_d}x^{2n_d} where a_i is in Z[k]
    # we then want to append [a_3,...,a_n_d]^T onto M_d where each a_i 
    # is a 'polynomial in kappa' 

    # we want to replace the monomials with precomputed values from reduction cashe

        output = 0
        for mo, co in zip(poly.monomials(), poly.coefficients()):
            exps = list(mo.exponents()[0])
    # gives exponent vector of monomial:
    # something like (x^4y^3k^2) --> [4,3,0,2]
   
            exps[0] = exps[0]//2 + m
            exps[1] = exps[1]//2
            output += co * reduction_cache[exps[0]+exps[1]][min(exps[0],exps[1])]
        content = output.content()
        output = reduce_int(content) * output // content
        ## 
        rows.append([output.coefficient({x: i}) for i in range(6, 2 * n_d + 1, 2)])

        tot += 1
        print(f"{tot}, ", end='', flush=True)
M = Matrix(S, rows[::-1])
row_count = M.nrows()
col_count = M.ncols()
M_square = M.submatrix(0, 0, col_count, col_count)

# PRIMITIVE POLYNOMIAL: over a finite field (size p), it is a monic irreducible 
# polynomial of degree m whos smallest positive integer e such that p(x) divides x^e-1 is p^m-1

# irreducible:a non-constant polynomial is irreducible over F if its coefficients belong 
# to F and it cannot be factored into the product of two non-constant polynomials with coefficients in F.

# end of lloyed chunk==================================================================================

# the following code is for ignoring kappa. Note that this became very easy because previously the author consider
# kappa as a variable, so he storage the matrix as a polynomial with kappa, so the whole functions are defined on 
# the ring Z[k]. However now we only consider k = 0, i.e. the constant term of Z[k], then Z[k] just became Z, and there
# is alot of step about delate bad kappa polynomial just gone. However I'm not sure if my code is correct because I just
# delate a lot. so maybe need debug later when we combine all the code
# the goal of this step is to check if the matrix is linearly independent. Remember that we need all polynomial to tau variant subset. so if not full rank = not all linearly independent = not all othogonal
det = M_square.det() # just take abs value since we don't want to calculate gcd(a,-b)
det = reduce_int(det) 
# don't need to do reduce poly since we no longer have a polynomial ring and the bad cases of kappa polunomial
# still need reduce_int since we don't want to consider small prime. (according to Daniel). not sure if we can delate that
gcd_all = det # gcd(det,det) = det 

for replace_row in range(lcm(col_count,row_count - col_count)): # pick every maximal minor
    M_copy = Matrix(M_square) # copy the matrix to avoid distroy it
    M_copy[replace_row % col_count, :] = M[(replace_row % (row_count - col_count)) + col_count, :] # replace some row to change the matrix, then we get different maximal minor
    det_i = M_copy.det() # calculate det
    det_i = reduce_int(det_i) # delate influence of small prime
    gcd_all = gcd(gcd_all, det_i) # gcd(a,b,c) = gcd(gcd(a,b),c)
    if det_i != 0:
        print (det_i)
        gcd_all = abs(reduce_int(det_i))
    if gcd_all == 1:
        print ("success") # print success if matrix is full rank under most of prime
        break



0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100


Computing reductions of 20 eigenvectors: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 

In [3]:
print(det.parent())

Integer Ring


In [4]:
print("raw det =", M_square.det())
print("after reduce =", det)

raw det = 0
after reduce = 0


In [7]:
for replace_row in range(10):
    M_copy = Matrix(M_square)
    M_copy[replace_row % col_count, :] = M[(replace_row % (row_count - col_count)) + col_count, :]
    print(M_copy.det())

0
0
0
0
0
0
0
0
0
0


In [8]:
print(M)

[-1148719149265902998389745896  -121287515447415426928229164   -12438432175852239642684304    -1194818731987521399510170     -104298570413198613828112       -8024253513536947020140        -523563362234564986984         -27369542816274899870          -1041219647334241280            -24083472675617568              -258122223816400                 -757973170048                             0                             0                             0                             0                             0                             0]
[  428370910074393266182296388    44494665866835596015370310     4436103724726976934120296      407615195732586657102709       33275374667492472116440        2320289256223929427270         131631257538243149012           5697978007167689655            170920812734841856              2808744189266960                17387556716040                   15949615544                             0                             0                             0        

In [11]:
print(M_square)
print("rank =", M_square.rank())

[-1148719149265902998389745896  -121287515447415426928229164   -12438432175852239642684304    -1194818731987521399510170     -104298570413198613828112       -8024253513536947020140        -523563362234564986984         -27369542816274899870          -1041219647334241280            -24083472675617568              -258122223816400                 -757973170048                             0                             0                             0                             0                             0                             0]
[  428370910074393266182296388    44494665866835596015370310     4436103724726976934120296      407615195732586657102709       33275374667492472116440        2320289256223929427270         131631257538243149012           5697978007167689655            170920812734841856              2808744189266960                17387556716040                   15949615544                             0                             0                             0        

NotImplementedError: Ideal Ideal (-1148719149265902998389745896, 428370910074393266182296388) of Multivariate Polynomial Ring in x, y, z over Integer Ring not principal
Echelon form not implemented over 'Multivariate Polynomial Ring in x, y, z over Integer Ring'.

In [15]:
Fp = GF(101)
M_mod = M_square.change_ring(Fp)

print("rank mod p =", M_mod.rank())

rank mod p = 12


In [37]:
print("rows =", M.nrows())
print("cols =", M.ncols())

for i in range(5):
    print(M[i])


rows = 20
cols = 18
(-1148719149265902998389745896, -121287515447415426928229164, -12438432175852239642684304, -1194818731987521399510170, -104298570413198613828112, -8024253513536947020140, -523563362234564986984, -27369542816274899870, -1041219647334241280, -24083472675617568, -258122223816400, -757973170048, 0, 0, 0, 0, 0, 0)
(428370910074393266182296388, 44494665866835596015370310, 4436103724726976934120296, 407615195732586657102709, 33275374667492472116440, 2320289256223929427270, 131631257538243149012, 5697978007167689655, 170920812734841856, 2808744189266960, 17387556716040, 15949615544, 0, 0, 0, 0, 0, 0)
(-79160013198252284231488225, -8117030363271559831783786, -790764049575669212558419, -69914165204124801772556, -5360821514948296105120, -337682000442206684872, -16247618476277520400, -546098244940550580, -12041736337808784, -129061111908200, -378986585024, 0, 0, 0, 0, 0, 0, 0)
(57965607977000379040988921, 5894864996753394791339450, 565425242517080231804427, 48632436146186342584

In [28]:
print(M.submatrix(0,0,5,5).det())
print(M.submatrix(3,0,5,5).det())

274035995384410632103780243616852196990087743459002209441915637228635078311387445208956456795562850677161984
222416658279061477635992064020063823804887550582046113749686569199708105503498988122348270947519692800


Problem: they are not full rank now ???

In [36]:
print(col_count)

18


In [38]:
Fp = GF(101)
print(M.change_ring(Fp).rank())

12


In [39]:
for p in [101,103,107,109]:
    print(p, M.change_ring(GF(p)).rank())

101 12
103 12
107 12
109 12
